<a href="https://colab.research.google.com/github/Shreyash007/CS6910-Assignment3/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!yes | wget "https://drive.google.com/file/d/1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw/view?usp=sharing"

--2023-05-21 12:52:13--  https://drive.google.com/file/d/1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.130.138, 74.125.130.100, 74.125.130.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.130.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  73.93K  --.-KB/s    in 0.01s   

2023-05-21 12:52:14 (6.42 MB/s) - ‘view?usp=sharing’ saved [75702]



In [2]:
!nvidia-smi

Sun May 21 12:52:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


# LOADING DATA FROM CSV FILES

In [4]:
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep=',',header=None,names=["en","ma",""],skip_blank_lines=True,index_col=None)
    data = data[data['en'].notna()]
    data = data[data['ma'].notna()]
    data = data[['en','ma']]
    return data

In [5]:

train = load_data("/content/drive/MyDrive/aksharantar_sampled/aksharantar_sampled/mar/mar_train.csv")
val = load_data("/content/drive/MyDrive/aksharantar_sampled/aksharantar_sampled/mar/mar_valid.csv")
test = load_data("/content/drive/MyDrive/aksharantar_sampled/aksharantar_sampled/mar/mar_test.csv")
print(test)
#below lists are used to save values in CSV again
test_en=list(test['en'])
test_ma=list(test['ma'])
print(test_en)
print(test_ma)
print(len(train))
print(len(test))
print(len(val))

              en         ma
0        heetler      हिटलर
1         kshama      क्षमा
2         jinkta     जिंकता
3        kaushik      कौशिक
4      jaadoogar     जादूगर
...          ...        ...
4091     devicha     देवीचा
4092      beebee       बेबी
4093    sukshama    सुक्ष्म
4094  julymadhye  जुलैमध्ये
4095      robins    रॉबिन्स

[4096 rows x 2 columns]
['heetler', 'kshama', 'jinkta', 'kaushik', 'jaadoogar', 'chaalavanyaachaa', 'samaajkaaryaat', 'thandine', 'maalmtaadharkaanmdhey', 'tukdoji', 'shivsainikasathi', 'subojitne', 'poshanasathi', 'hotee', 'ushmaa', 'tondawar', 'mele', 'annapolis', 'bombay', 'aavadtya', 'adhyaksheey', 'shetkaryaanchyaabaabat', 'barai', 'sangneamulay', 'sankate', 'mets', 'mulam', 'tinada', 'antarvastrapaasun', 'durgech', 'mulachaa', 'harananche', 'kashtache', 'talhta', 'eeseen', 'dharmshikshanaache', 'shheffield', 'loid', 'rugnanvar', 'anuvadasathee', 'babhadgav', 'karanehee', 'sampalyaane', 'ekalaa', 'chakreewadalacha', 'vibhagiya', 'doghaannihi', 'shiks

#TOKENIZE DATASET

In [6]:
def unique_tokenize(data):
    english = train['en'].values
    marathi = train['ma'].values
    english_tokens = set()
    marathi_tokens = set()
    
    for x,y in zip(english,marathi):
        for ch in x:
            english_tokens.add(ch)
        for ch in y:
            marathi_tokens.add(ch)
    english_tokens = sorted(list(english_tokens))
    marathi_tokens = sorted(list(marathi_tokens))
    return marathi_tokens , english_tokens
marathi_tokens , english_tokens = unique_tokenize(train)
print(english_tokens)
print(len(marathi_tokens))

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
63


#TOKEN MAP

In [7]:
#REWRITE AGAIN
def tokenize_map(hindi_tokens , english_tokens):
    english_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
    marathi_token_map = dict([(ch,i+1) for i,ch in enumerate(marathi_tokens)])
    reverse_marathi_token_map = dict([(i+1,ch) for i,ch in enumerate(marathi_tokens)])
    #Adding blank space

    marathi_token_map[" "] = 0
    english_token_map[" "] = 0
    #addin BOS and EOS token 
    marathi_token_map[';']=65
    marathi_token_map['.']=66
    english_token_map[';']=27
    english_token_map['.']=28
    marathi_token_map['<unk>']=64
    
    reverse_marathi_token_map[64]='<unk>'
    reverse_marathi_token_map[65]=';'
    reverse_marathi_token_map[66]='.'
    reverse_marathi_token_map[0]=''

    return marathi_token_map, english_token_map,reverse_marathi_token_map

mar_token_map, eng_token_map,reverse_marathi_token_map = tokenize_map(marathi_tokens , english_tokens)
print(mar_token_map)
print(reverse_marathi_token_map)
print(eng_token_map)

print('mar:',len(mar_token_map))
print('eng:',len(eng_token_map))

{'ँ': 1, 'ं': 2, 'ः': 3, 'अ': 4, 'आ': 5, 'इ': 6, 'ई': 7, 'उ': 8, 'ऊ': 9, 'ऋ': 10, 'ऍ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'च': 21, 'छ': 22, 'ज': 23, 'झ': 24, 'ञ': 25, 'ट': 26, 'ठ': 27, 'ड': 28, 'ढ': 29, 'ण': 30, 'त': 31, 'थ': 32, 'द': 33, 'ध': 34, 'न': 35, 'प': 36, 'फ': 37, 'ब': 38, 'भ': 39, 'म': 40, 'य': 41, 'र': 42, 'ल': 43, 'ळ': 44, 'व': 45, 'श': 46, 'ष': 47, 'स': 48, 'ह': 49, '़': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ो': 61, 'ौ': 62, '्': 63, ' ': 0, ';': 65, '.': 66, '<unk>': 64}
{1: 'ँ', 2: 'ं', 3: 'ः', 4: 'अ', 5: 'आ', 6: 'इ', 7: 'ई', 8: 'उ', 9: 'ऊ', 10: 'ऋ', 11: 'ऍ', 12: 'ए', 13: 'ऐ', 14: 'ऑ', 15: 'ओ', 16: 'औ', 17: 'क', 18: 'ख', 19: 'ग', 20: 'घ', 21: 'च', 22: 'छ', 23: 'ज', 24: 'झ', 25: 'ञ', 26: 'ट', 27: 'ठ', 28: 'ड', 29: 'ढ', 30: 'ण', 31: 'त', 32: 'थ', 33: 'द', 34: 'ध', 35: 'न', 36: 'प', 37: 'फ', 38: 'ब', 39: 'भ', 40: 'म', 41: 'य', 42: 'र', 43: 'ल', 44: 'ळ', 45: 'व', 46: '

#MAXIMUM WORD LENGTH THAT ARE PRESENT IN THE DATASET




In [8]:

x = test['en'].values
y = test['ma'].values
x=';'+ x + '.'
y = ';'+y+'.'
#Getting max length
max_eng_len = max([len(i) for i in x])
max_mar_len = max([len(i) for i in y])
print(max_eng_len,max_mar_len)

30 22


# ONE HOT ENCODING/EMBEDDING OUR INPUT



In [9]:

import torch
#unknown token present in validation set as 'r.(in marathi)'
unknown_token=64
def process(data):
    x,y = data['en'].values, data['ma'].values
    x = ";" + x + "."
    y = ";" + y + "."
    print(x[0:3])
    print(y[0:3]) 
    
    a = torch.zeros((len(x),max_eng_len),dtype=torch.int64)
    print(a.shape)
    
    b = torch.zeros((len(y),max_eng_len),dtype=torch.int64)
    
    data=[]
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = eng_token_map[ch]

        #a[i,j+1:] = eng_token_map[" "]
        for j,ch in enumerate(yy):
            if ch in mar_token_map: 
             b[i,j] = mar_token_map[ch]
            else:
              b[i,j]= unknown_token


    data = [(a[i], b[i]) for i in range(len(x))]
    print(a.shape)
    print(b.shape)
    return data

train_process=process(train)
val_process=process(val)
test_process=process(test)
#print(train_process.shape)
print('\n')
print('num of rows:',len(train_process))
print('num of columns:',len(train_process[0]))
print(train_process[0][1])

[';fusharun.' ';bhulthapana.' ';vhayaki.']
[';फुशारुन.' ';भूलथापाना.' ';व्हायकी.']
torch.Size([51200, 30])
torch.Size([51200, 30])
torch.Size([51200, 30])
[';garvyabarobarach.' ';reo.' ';sangrahalaye.']
[';गारव्याबरोबरच.' ';रियो.' ';संग्रहालये.']
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])
[';heetler.' ';kshama.' ';jinkta.']
[';हिटलर.' ';क्षमा.' ';जिंकता.']
torch.Size([4096, 30])
torch.Size([4096, 30])
torch.Size([4096, 30])


num of rows: 51200
num of columns: 2
tensor([65, 37, 54, 46, 51, 42, 54, 35, 66,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [10]:
#Used later for reading the words

from torch import tensor
def reverse_tokenize(data):
  data=[int(i) for i in data]
  predicted_seq = [reverse_marathi_token_map[idx] for idx in data]
  predicted_seq=''.join(predicted_seq)
  return predicted_seq
  

;फुशारुन.


#DATA LOADER

In [11]:

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 16

PAD_IDX = 0
BOS_IDX = ';'
EOS_IDX = '.'
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

#WE ARE NOT USING GENERATE BATCH FUNCTION
def generate_batch(data_batch):
  ma_batch, en_batch = [], []
  for (en_item,ma_item) in data_batch:
    #ma_batch.append(torch.cat([torch.tensor(mar_token_map[BOS_IDX]), ma_item, torch.tensor(mar_token_map[EOS_IDX])], dim=0))
    #en_batch.append(torch.cat([torch.tensor(eng_token_map[BOS_IDX]), en_item, torch.tensor(eng_token_map[EOS_IDX])], dim=0))
    #print(ma_item[0])
    ma_batch=torch.tensor(ma_item,dtype=torch.int64)
    en_batch=torch.tensor(en_item,dtype=torch.int64)
  
  return  en_batch.to(device),ma_batch.to(device)



train_iter = DataLoader(train_process, batch_size=BATCH_SIZE,
                        shuffle=False)
valid_iter = DataLoader(val_process, batch_size=BATCH_SIZE,
                        shuffle=False)
test_iter = DataLoader(test_process, batch_size=BATCH_SIZE,
                       shuffle=False)
print(len(train_iter))
print(len(test_iter))
print(len(valid_iter))

3200
256
256


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import random
from collections import Counter
# Set random seed for reproducibility
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#os.environ['TORCH_USE_CUDA_DSA'] = '1'


In [ ]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 
import wandb

#ATTENTION MODEL

In [22]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        #print('inp. dim')
        #print(self.input_dim)
        #print('emb. dim')
        #print(self.emb_dim)
        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
        

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:
        src = src.permute(1,0)
        #print(src)
        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)
        input = input.permute(1,0)
        embedded = self.dropout(self.embedding(input))
        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        #print(weighted_encoder_rep.shape)
        embedded = embedded.permute(1,0,2)
        
        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        #ATTENTION HEAT MAP METHOD
        #attentions = torch.zeros(max_len, batch_size, src.shape[1]).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        trg = trg.permute(1,0)
        output = trg[0,:]
        
        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(eng_token_map)
OUTPUT_DIM = len(mar_token_map)
ENC_EMB_DIM = 512
DEC_EMB_DIM = 512
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ATTN_DIM = 256
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

#added custom weights
model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,630,467 trainable parameters


# ATTENTION WANDB IMPLEMENTATION

In [ ]:
import math
import time
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'emb_dim': {
            'values': [128,256,512]
        },
        'hidden_layer_dim': {
            'values': [128,256,512]
        },        
        'attn_dim':{
            'values':[64,128,256]
        },
        'dropout':{
            'values':[0.3,0.5,0.6]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS6910-Assignment3")

def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'emb_dim':256,
      'hidden_layer_dim':256,
      'attn_dim':128,
      'dropout':0.6,
  }

  # Initialize a new wandb run
  wandb.init(project='CS6910-Assignment3', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'emb_dim:'+ str(wandb.config.emb_dim)+' ;hl:'+str(wandb.config.hidden_layer_dim)+ ' ;attn_dim:'+str(wandb.config.attn_dim)+ ' ;dropout:'+str(wandb.config.dropout)
  config = wandb.config
  emb_dim = config.emb_dim
  hidden_layer_dim = config.hidden_layer_dim
  attn_dim = config.attn_dim
  dropout = config.dropout
  # Model training here

  INPUT_DIM = len(eng_token_map)
  OUTPUT_DIM = len(mar_token_map)

  enc = Encoder(INPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout)

  attn = Attention(hidden_layer_dim, hidden_layer_dim, attn_dim)

  dec = Decoder(OUTPUT_DIM, emb_dim, hidden_layer_dim, hidden_layer_dim, dropout, attn)

  model = Seq2Seq(enc, dec, device).to(device)
  model.apply(init_weights)
  optimizer = optim.Adam(model.parameters())
  criterion = nn.CrossEntropyLoss()

  model.train()

  train_epoch_loss = 0
  train_epoch_acc = 0
  val_epoch_loss = 0
  val_epoch_acc = 0
  
  N_EPOCHS = 10
  CLIP = 1

  for epoch in range(N_EPOCHS):

  #TRAINING BLOCK
    model.train()
    for _, (src, trg) in enumerate(train_iter):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg.permute(1,0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)
        
        # calculate accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
        train_correct = preds[non_pad_elements] == trg[non_pad_elements]
        train_epoch_acc += train_correct.sum().item() / len(non_pad_elements)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

        optimizer.step()

        train_epoch_loss += loss.item()


    train_epoch_loss = train_epoch_loss / len(train_iter)
    train_epoch_acc = train_epoch_acc / len(train_iter)


    #EVALUATION MODE
    model.eval()
    
    with torch.no_grad():

        for _, (src, trg) in enumerate(valid_iter):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            val_epoch_loss += loss.item()
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            val_correct = preds[non_pad_elements] == trg[non_pad_elements]
            val_epoch_acc += val_correct.sum().item() / len(non_pad_elements)

    val_epoch_loss = val_epoch_loss / len(valid_iter)
    val_epoch_acc = val_epoch_acc / len(valid_iter)

    print(f'Epoch: {epoch+1:02} |Train Loss: {train_epoch_loss:.3f} | Train accuracy: {train_epoch_acc:.3f}|Val. Loss: {val_epoch_loss:.3f} | Val accuracy: {val_epoch_acc:.3f}')
    wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
    #emptying the cache after one complete run
    if epoch==N_EPOCHS-1:
            torch.cuda.empty_cache()


#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=120)

# VANILLA SEQ2SEQ MODEL

In [23]:

criterion = nn.CrossEntropyLoss()
import math
import time
import numpy as np



def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0
    epoch_acc = 0
    epoch_word_acc = 0
    total_chars = 0
    correct_chars = 0
    total_words = 0
    correct_words = 0


    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()

        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg.permute(1,0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)
        
        # calculate character level accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
        correct = preds[non_pad_elements] == trg[non_pad_elements]
        epoch_acc += correct.sum().item() / len(non_pad_elements)
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0
    epoch_acc = 0
    epoch_word_acc = 0
    total_chars = 0
    correct_chars = 0
    total_words = 0
    correct_words = 0
    solution=[]
    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            epoch_loss += loss.item()
            
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            correct = preds[non_pad_elements] == trg[non_pad_elements]
            epoch_acc += correct.sum().item() / len(non_pad_elements)

            b=np.zeros((16,29))
            for i in range(16):
              for j in range (29):
                 b[i][j]=preds[16*j+i]
            
            for i in range(16):
              solution.append(reverse_tokenize(b[i]))


    return epoch_loss / len(iterator), epoch_acc / len(iterator), solution


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss,train_acc = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss,val_acc,solution_valid = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_acc:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val accuracy: {val_acc:.3f}')
    print(solution_valid[0])

test_loss,test_accuracy,solution_test_att = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy:.3f} |')

Epoch: 01 | Time: 3m 35s
	Train Loss: 0.845 | Train accuracy: 0.386
	 Val. Loss: 0.590 | Val accuracy: 0.492
गर्वायाबरोबररच.
Epoch: 02 | Time: 3m 35s
	Train Loss: 0.333 | Train accuracy: 0.739
	 Val. Loss: 0.445 | Val accuracy: 0.647
गरव्याबरोबरच.
Epoch: 03 | Time: 3m 38s
	Train Loss: 0.234 | Train accuracy: 0.817
	 Val. Loss: 0.415 | Val accuracy: 0.687
गरव्याबरोबरच.
Epoch: 04 | Time: 3m 38s
	Train Loss: 0.207 | Train accuracy: 0.839
	 Val. Loss: 0.410 | Val accuracy: 0.699
गरव्याबरोबरच.
Epoch: 05 | Time: 3m 37s
	Train Loss: 0.193 | Train accuracy: 0.851
	 Val. Loss: 0.413 | Val accuracy: 0.706
गरव्याबरोबरच.
Epoch: 06 | Time: 3m 33s
	Train Loss: 0.182 | Train accuracy: 0.860
	 Val. Loss: 0.403 | Val accuracy: 0.705
गरव्याबरोबरच.
Epoch: 07 | Time: 3m 29s
	Train Loss: 0.174 | Train accuracy: 0.866
	 Val. Loss: 0.428 | Val accuracy: 0.712
गरव्याबरोबरच.
Epoch: 08 | Time: 3m 29s
	Train Loss: 0.173 | Train accuracy: 0.868
	 Val. Loss: 0.426 | Val accuracy: 0.704
गारव्याबरोबरच.
Epoch: 09 | T

#VANILLA SEQ2SEQ MODEL ON TEST DATASET




In [18]:
# Define the vocabulary of English and Devanagari characters

# Define the maximum sequence lengths for input and output sequences
MAX_LEN_EN = 30
MAX_LEN_MA = 30

# Define the start and end of sequence tokens
BOS_token = 28
EOS_token = 29


class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers=1 , cell_type="lstm", p=0.5):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type=cell_type
        self.dropout=nn.Dropout(p)
        self.embedding = nn.Embedding(input_size,embedding_size)

        if cell_type == "gru":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers,dropout=p)
        elif cell_type == "lstm":
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers,dropout=p)
        elif cell_type == "rnn":
            self.rnn = nn.RNN(input_size, hidden_size, num_layers,dropout=p)
        else:
            raise ValueError("Invalid cell type selected: {}".format(cell_type))

    def forward(self, x):
        
        x=x.permute(1,0)
        embedding=self.dropout(self.embedding(x))
        if self.cell_type=="lstm":
         outputs,(hidden,cell)=self.rnn(embedding)
        else:
           hidden, cell = self.rnn(embedding)
        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1, cell_type="lstm",p=0.5):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout=nn.Dropout(p)
        self.embedding=nn.Embedding(input_size, embedding_size)
        self.cell_type=cell_type

        self.output_size = output_size
        if cell_type == "gru":
            self.rnn = nn.GRU(output_size, hidden_size, num_layers)
        elif cell_type == "lstm":
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers,dropout=p)
        elif cell_type == "rnn":
            self.rnn = nn.RNN(output_size, hidden_size, num_layers)
        else:
            raise ValueError("Invalid cell type selected: {}".format(cell_type))
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden,cell):
       # shape of x: (N) but we want (1,N)
       x=x.unsqueeze(0)

       embedding= self.dropout(self.embedding(x))
       #embedding shape: (1,N,hidden_size)
       
       if self.cell_type=="lstm":
        outputs,(hidden,cell) = self.rnn(embedding, (hidden,cell))
       else:
         outputs, hidden = self.rnn(embedding, hidden)
       #shape of outputs: (1,N,hidden_size)
       

       predictions=self.fc(outputs)
       #shape of predictions: (1,N,length_of_vocab)

       predictions = predictions.squeeze(0)
       
       if self.cell_type=="lstm":
         return predictions, hidden, cell
       else:
         return predictions, hidden


class Seq2Seq(nn.Module):
    def __init__(
        self,encoder,decoder,cell_type="lstm"):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[0]
        
        target_len = target.shape[1]
        target_vocab_size=self.decoder.output_size

        outputs=torch.zeros(target_len,batch_size,target_vocab_size).to(device)
        
        cell,hidden=self.encoder(source)
        
        target = target.permute(1,0)
        x = target[0,:]
        for t in range(1,target_len):
          if self.cell_type=="lstm":
           output,hidden,cell =self.decoder(x,hidden,cell)
          else:
            output, hidden = self.decoder(x,hidden,cell)

          outputs[t] = output
          
          best_guess = output.argmax(1)

          x= target[t] if random.random() < teacher_force_ratio else best_guess
        return outputs


In [19]:
import numpy as np
num_epochs=10
learning_rate=0.001

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(eng_token_map)
input_size_decoder=len(mar_token_map)
output_size=len(mar_token_map)

encoder_embedding_size=29
decoder_embedding_size=67

cell_type="gru"
hidden_size=256
num_layers=4
enc_dropout=0.2
dec_dropout=0.2
#input_size, embedding_size, hidden_size, num_layers=1 , cell_type="lstm", p=0.5
encoder_net=Encoder(input_size_encoder,encoder_embedding_size,hidden_size,num_layers,cell_type,p=enc_dropout).to(device)
decoder_net=Decoder(input_size_decoder,decoder_embedding_size,hidden_size,output_size,num_layers,cell_type,p=dec_dropout).to(device)
model=Seq2Seq(encoder_net,decoder_net,cell_type).to(device)
criterion = nn.CrossEntropyLoss()
import math
import time

optimizer = optim.Adam(model.parameters(),lr=learning_rate)
def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0
    epoch_acc = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()

        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg.permute(1,0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)
        
        # calculate accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
        correct = preds[non_pad_elements] == trg[non_pad_elements]
        epoch_acc += correct.sum().item() / len(non_pad_elements)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()
    solution=[]
    epoch_loss = 0
    epoch_acc = 0
    
    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            epoch_loss += loss.item()
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            b=np.zeros((16,29))
            for i in range(16):
              for j in range (29):
                 b[i][j]=preds[16*j+i]
            
            for i in range(16):
              solution.append(reverse_tokenize(b[i]))
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            correct = preds[non_pad_elements] == trg[non_pad_elements]
            epoch_acc += correct.sum().item() / len(non_pad_elements)
            

    return epoch_loss / len(iterator), epoch_acc / len(iterator),solution


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss,train_acc = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss,val_acc,solution = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_acc:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val accuracy: {val_acc:.3f}')
    print(solution[0])


test_loss,test_accuracy,solution_test = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy:7.3f} |')



Epoch: 01 | Time: 2m 28s
	Train Loss: 0.732 | Train accuracy: 0.470
	 Val. Loss: 0.509 | Val accuracy: 0.578
गर्व्याबरोबरच.
Epoch: 02 | Time: 2m 30s
	Train Loss: 0.339 | Train accuracy: 0.732
	 Val. Loss: 0.466 | Val accuracy: 0.633
गर्व्याबरोबरच.
Epoch: 03 | Time: 2m 30s
	Train Loss: 0.274 | Train accuracy: 0.783
	 Val. Loss: 0.433 | Val accuracy: 0.668
गरव्याबरोबरच.
Epoch: 04 | Time: 2m 29s
	Train Loss: 0.248 | Train accuracy: 0.803
	 Val. Loss: 0.407 | Val accuracy: 0.687
गरव्याबरोबरच.
Epoch: 05 | Time: 2m 30s
	Train Loss: 0.231 | Train accuracy: 0.817
	 Val. Loss: 0.427 | Val accuracy: 0.693
गरव्याबरोबरच.
Epoch: 06 | Time: 2m 29s
	Train Loss: 0.221 | Train accuracy: 0.825
	 Val. Loss: 0.419 | Val accuracy: 0.698
गर््यावरोबरच.
Epoch: 07 | Time: 2m 30s
	Train Loss: 0.217 | Train accuracy: 0.829
	 Val. Loss: 0.417 | Val accuracy: 0.703
गरव्याबरोबरच.
Epoch: 08 | Time: 2m 31s
	Train Loss: 0.213 | Train accuracy: 0.832
	 Val. Loss: 0.416 | Val accuracy: 0.702
गरव्याबरोबरच.
Epoch: 09 | Ti

In [ ]:

!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 
import wandb

# WANDB IMPLEMENTATION

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'lr': {
            'values': [0.001,0.0001]
        },
        'layer_size': {
            'values': [1,2,3,4]
        },        
        'cell_type':{
            'values':['rnn','lstm','gru']
        },
        'dropout':{
            'values':[0,0.2,0.4,0.6]
        },
        'hidden_layers':{
            'values':[64,128,256]
        },
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS6910-Assignment3")

def sweep_train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'lr':0.0001,
      'layer_size':4,
      'cell_type':'lstm',
      'dropout':0.4,
      'hidden_layers':128,
  }

  # Initialize a new wandb run
  wandb.init(project='CS6910-Assignment3', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'cell:'+ str(wandb.config.cell_type)+' ;lr:'+str(wandb.config.lr)+ ' ;layer_size:'+str(wandb.config.layer_size)+ ' ;dropout:'+str(wandb.config.dropout)+' ;hidden:'+str(wandb.config.hidden_layers)
  config = wandb.config
  lr = config.lr
  layer_size = config.layer_size
  cell_type = config.cell_type
  hidden_layers = config.hidden_layers  
  dropout = config.dropout
  # Model training here

  input_size_encoder = len(eng_token_map)
  output_size=input_size_decoder=len(mar_token_map)

  encoder_embedding_size=29
  decoder_embedding_size=67
  encoder_net=Encoder(input_size_encoder,encoder_embedding_size,hidden_layers,layer_size,cell_type,p=dropout).to(device)
  decoder_net=Decoder(input_size_decoder,decoder_embedding_size,hidden_layers,output_size,layer_size,cell_type,p=dropout).to(device)
  model=Seq2Seq(encoder_net,decoder_net,cell_type).to(device)
  criterion = nn.CrossEntropyLoss()
  import math
  import time
 
  optimizer = optim.Adam(model.parameters(),lr=lr)
  model.train()

  train_epoch_loss = 0
  train_epoch_acc = 0
  val_epoch_loss = 0
  val_epoch_acc = 0
  
  N_EPOCHS = 10
  CLIP = 1

  for epoch in range(N_EPOCHS):

  #TRAINING BLOCK
    model.train()
    for _, (src, trg) in enumerate(train_iter):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()

        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg.permute(1,0)
        trg = torch.reshape(trg[1:], (-1,))
        loss = criterion(output, trg)
        
        # calculate accuracy
        preds = torch.argmax(output, dim=1)
        non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
        train_correct = preds[non_pad_elements] == trg[non_pad_elements]
        train_epoch_acc += train_correct.sum().item() / len(non_pad_elements)
        loss.backward()

        #READ IF WE CAN PASS CLIP AS A HYPERPARAMETER 
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

        optimizer.step()

        train_epoch_loss += loss.item()


    train_epoch_loss = train_epoch_loss / len(train_iter)
    train_epoch_acc = train_epoch_acc / len(train_iter)


    #EVALUATION MODE
    model.eval()
    
    with torch.no_grad():

        for _, (src, trg) in enumerate(valid_iter):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg.permute(1,0)
            trg = torch.reshape(trg[1:], (-1,))

            loss = criterion(output, trg)

            val_epoch_loss += loss.item()
            # calculate accuracy
            preds = torch.argmax(output, dim=1)
            non_pad_elements = (trg != 0).nonzero(as_tuple=True)[0]
            val_correct = preds[non_pad_elements] == trg[non_pad_elements]
            val_epoch_acc += val_correct.sum().item() / len(non_pad_elements)

    val_epoch_loss = val_epoch_loss / len(valid_iter)
    val_epoch_acc = val_epoch_acc / len(valid_iter)

    print(f'Epoch: {epoch+1:02} |Train Loss: {train_epoch_loss:.3f} | Train accuracy: {train_epoch_acc:.3f}|Val. Loss: {val_epoch_loss:.3f} | Val accuracy: {val_epoch_acc:.3f}')
    wandb.log({"train_loss":train_epoch_loss,"train_accuracy": train_epoch_acc,"val_loss":val_epoch_loss,"val_accuracy":val_epoch_acc},)
    #emptying the cache after one complete run
    if epoch==N_EPOCHS-1:
            torch.cuda.empty_cache()


#RUNNING THE SWEEP
wandb.agent(sweep_id, function=sweep_train, count=120)


Error in callback <function _WandbInit._resume_backend at 0x7ffaa83481f0> (for pre_run_cell):


BrokenPipeError: ignored

Create sweep with ID: t1x3tqqp
Sweep URL: https://wandb.ai/shreyashgadgil007/CS6910-Assignment3/sweeps/t1x3tqqp


wandb: Agent Starting Run: rfsh9ynk with config:
wandb: 	cell_type: rnn
wandb: 	dropout: 0
wandb: 	hidden_layers: 256
wandb: 	layer_size: 1
wandb: 	lr: 0.0001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch: 01 |Train Loss: 1.255 | Train accuracy: 0.168|Val. Loss: 1.302 | Val accuracy: 0.102
Epoch: 02 |Train Loss: 1.172 | Train accuracy: 0.197|Val. Loss: 1.276 | Val accuracy: 0.113
Epoch: 03 |Train Loss: 1.155 | Train accuracy: 0.205|Val. Loss: 1.221 | Val accuracy: 0.108
Epoch: 04 |Train Loss: 1.145 | Train accuracy: 0.210|Val. Loss: 1.209 | Val accuracy: 0.108
Epoch: 05 |Train Loss: 1.139 | Train accuracy: 0.213|Val. Loss: 1.212 | Val accuracy: 0.108
Epoch: 06 |Train Loss: 1.132 | Train accuracy: 0.216|Val. Loss: 1.253 | Val accuracy: 0.092
Epoch: 07 |Train Loss: 1.130 | Train accuracy: 0.217|Val. Loss: 1.252 | Val accuracy: 0.106
Epoch: 08 |Train Loss: 1.125 | Train accuracy: 0.220|Val. Loss: 1.289 | Val accuracy: 0.115
Epoch: 09 |Train Loss: 1.122 | Train accuracy: 0.222|Val. Loss: 1.216 | Val accuracy: 0.108
Epoch: 10 |Train Loss: 1.118 | Train accuracy: 0.224|Val. Loss: 1.248 | Val accuracy: 0.103


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: i8us8s8z with config:
wandb: 	cell_type: gru
wandb: 	dropout: 0.6
wandb: 	hidden_layers: 256
wandb: 	layer_size: 4
wandb: 	lr: 0.001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch: 01 |Train Loss: 0.894 | Train accuracy: 0.360|Val. Loss: 0.610 | Val accuracy: 0.470
Epoch: 02 |Train Loss: 0.524 | Train accuracy: 0.599|Val. Loss: 0.528 | Val accuracy: 0.558
Epoch: 03 |Train Loss: 0.436 | Train accuracy: 0.663|Val. Loss: 0.500 | Val accuracy: 0.592
Epoch: 04 |Train Loss: 0.395 | Train accuracy: 0.694|Val. Loss: 0.486 | Val accuracy: 0.611


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7ffaa8348280> (for post_run_cell):


BrokenPipeError: ignored

# SAVING OUTPUT IN CSV FILES AND WORD LEVEL ACCURACY CALCULATION

In [24]:
#removing end of sentence token
solution_test_att = [word.replace('.', '') for word in solution_test_att]
solution_test = [word.replace('.', '') for word in solution_test]
def calculate_accuracy(list1, list2):
    total_words = len(list1)
    correct_words = 0

    for word1, word2 in zip(list1, list2):
        if word1 == word2:
            correct_words += 1

    accuracy = correct_words / total_words * 100
    return accuracy

vanilla_seq2seq_accuracy=calculate_accuracy(test_ma,solution_test)
attention_seq2seq_accuracy=calculate_accuracy(test_ma,solution_test_att)
print('vanilla_seq2seq_Acc.:',vanilla_seq2seq_accuracy)
print('attention_seq2seq_Acc.:',attention_seq2seq_accuracy)

vanilla_seq2seq_Acc.: 29.98046875
attention_seq2seq_Acc.: 32.3486328125


In [25]:
import csv

file_path = '/content/drive/MyDrive/Predictions_attention.csv'
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the column headers
    writer.writerow(['English', 'Marathi_translation', 'Attention_seq2seq_pred'])

    # Write the data rows
    for row in zip(test_en, test_ma, solution_test_att):
        writer.writerow(row)

In [26]:
import csv
file_path = '/content/drive/MyDrive/Predictions_vanilla.csv'
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the column headers
    writer.writerow(['English', 'Marathi_translation', 'Vanilla_seq2seq_pred'])

    # Write the data rows
    for row in zip(test_en, test_ma, solution_test):
        writer.writerow(row)
